<a href="https://colab.research.google.com/github/youkiti/ARE/blob/postspinal-hypotension/TA_screening_zeroshot_imai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import glob
from pathlib import Path
import math
from google.colab import files

In [ ]:
folderPath = "PATH"
os.chdir(folderPath)

In [ ]:
df = pd.read_csv('articles.csv')

In [ ]:
df1 = df[~df["notes"].isna()]

In [ ]:
print(f"全体{df1.shape[0]}")

全体50


In [ ]:
df1.loc[:, 'contains_included'] = df1['notes'].str.contains('Included')
print(f"組入の数: {len(df1[df1['contains_included'] == True])}")

組入の数: 17


<ipython-input-7-77ba1064df38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[:, 'contains_included'] = df1['notes'].str.contains('Included')


In [ ]:
pd.set_option('display.max_colwidth', 100)
print(f'不一致の数: {df1[df1["contains_included"] == True]["notes"].str.contains("Excluded").sum()}')

不一致の数: 3


不一致は組入扱いで進める

In [ ]:
#tiab列を作る
df1.loc[:, 'tiab'] = 'TITLE: ' + df1['title'].fillna('') + ' ABSTRACT: ' + df1['abstract'].fillna('')

openai apiの準備

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI
import json
from tqdm import tqdm
import time

#secretで設定したキー
from google.colab import userdata
API_KEY = userdata.get('opanai_API_key')
#渡す

os.environ['OPENAI_API_KEY'] = API_KEY

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get(API_KEY),
)

In [ ]:
def get_completion(system_prompt, prompt, tools):
    function_name = tools[0]["function"]["name"]
    chat_completion = client.chat.completions.create(
        messages=[
                {"role":"system",
                "content":system_prompt},
                {
                "role": "user",
                "content": prompt
                }],
            tools = tools,
            tool_choice =  {"type": "function", "function": {"name": function_name}},
            temperature = 0,
            model = "gpt-4o"
          )

    arguments_json_str = chat_completion.choices[0].message.tool_calls[0].function.arguments

    arguments_json = json.loads(arguments_json_str)

    return arguments_json

In [ ]:
#二値を返すtoolsを定義
tools = [{"type": "function",
    "function": {
        "name": "abstract_inclusion_decision",
        "description": "Determine if an abstract meets the criteria",
        "parameters": {
            "type": "object",
            "properties": {
                "judgement": {
                    "type": "boolean",
                    "description": "The judgement of the abstract"
                }
            }
        }
    }
    }]

ここで、組入・除外を指令

In [ ]:
system_prompt = """
You are conducting a systematic review.
Please determine if the user inputted abstract should be included based on the following criteria. If there is insufficient information to make a decision, the abstract should be included for further evaluation.

# Inclusion Criteria:
1. **Study Design:**
   - Randomized Controlled Trials (RCTs)

2. **Participants:**
   - Adult patients undergoing cesarean section under spinal anesthesia or combined spinal–epidural anesthesia (CSEA).
   - Includes both elective and non-elective (urgent and emergency) cesarean sections.

3. **Interventions:**
   - Studies must evaluate the effects of norepinephrine (noradrenaline) or phenylephrine administered via: bolus or infsion
   - Both therapeutic (administered after hypotension onset) and prophylactic (administered before hypotension onset) administrations are considered.

"""

In [ ]:
judgements = []

# Number of retries for failed API calls
max_retries = 3
retry_delay = 5  # seconds

for prompt in tqdm(df1["tiab"], desc="Processing Prompts"):
    for attempt in range(max_retries):
        try:
            request_payload = {"prompt": prompt, "tools": tools}
            # Call the get_completion function and extract the "judgement" from the result
            completion_result = get_completion(system_prompt, prompt, tools)
            judgement = completion_result.get("judgement", None)  # Use get to handle missing keys

            # Append the judgement to the list
            judgements.append(judgement)
            break  # Break out of the retry loop if successful
        except Exception as e:
            # Log the error or print it for debugging
            print(f"Error processing prompt (attempt {attempt+1}/{max_retries}): {prompt}")
            print(f"Error: {str(e)}")

            # Check if it's a specific JSON parsing error and handle it
            if "We could not parse the JSON body of your request" in str(e):
                print("Request payload:", json.dumps(request_payload, indent=2))  # Print the JSON payload
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds due to JSON parsing issue...")
                    time.sleep(retry_delay)
                else:
                    # Append a default value (e.g., None) to the list after max retries
                    judgements.append(None)
            else:
                # If it's a different error, append None and break the loop
                judgements.append(None)
                break

Processing Prompts: 100%|██████████| 50/50 [00:22<00:00,  2.17it/s]


In [ ]:
#くっつける
from sklearn.metrics import confusion_matrix

df1.loc[:,'judgement'] = judgements

# Convert 'judgement' to binary: True -> 1, False -> 0 うまく返ってこなかったものは組入扱いにする
df1['judgement'] = df1['judgement'].fillna(1).astype(int)

df1["contains_included"] = df1["contains_included"].astype(int)


# Calculate the confusion matrix
conf_matrix = confusion_matrix( df1['judgement'],df1['contains_included'], labels=[1, 0])

# Extract the components of the confusion matrix
tp, fp, fn, tn = conf_matrix.ravel()

# Calculate sensitivity and specificity
sensitivity = tp / (tp + fn)  # True Positive Rate
specificity = tn / (tn + fp)  # True Negative Rate

# Print the results
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

In [ ]:
df1.to_csv('articles_judgement.csv', index=False)

In [ ]:
#download
from google.colab import files
files.download('articles_judgement.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

判断がないものにラベルを付与  
False判断のものを除外  
Rayyanで使えるように列の編集  
エクスポート

In [ ]:
df2 = df[df["notes"].isna()]

In [ ]:
df2.shape

(1683, 21)

In [ ]:
#tiab列を作る
df2.loc[:, 'tiab'] = 'TITLE: ' + df2['title'].fillna('') + ' ABSTRACT: ' + df2['abstract'].fillna('')

<ipython-input-66-6f46269fbc0c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:, 'tiab'] = 'TITLE: ' + df2['title'].fillna('') + ' ABSTRACT: ' + df2['abstract'].fillna('')


In [ ]:
judgements = []

# Number of retries for failed API calls
max_retries = 3
retry_delay = 5  # seconds

for prompt in tqdm(df2["tiab"], desc="Processing Prompts"):
    for attempt in range(max_retries):
        try:
            request_payload = {"prompt": prompt, "tools": tools}
            # Call the get_completion function and extract the "judgement" from the result
            completion_result = get_completion(system_prompt2, prompt, tools)
            judgement = completion_result.get("judgement", None)  # Use get to handle missing keys

            # Append the judgement to the list
            judgements.append(judgement)
            break  # Break out of the retry loop if successful
        except Exception as e:
            # Log the error or print it for debugging
            print(f"Error processing prompt (attempt {attempt+1}/{max_retries}): {prompt}")
            print(f"Error: {str(e)}")

            # Check if it's a specific JSON parsing error and handle it
            if "We could not parse the JSON body of your request" in str(e):
                print("Request payload:", json.dumps(request_payload, indent=2))  # Print the JSON payload
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds due to JSON parsing issue...")
                    time.sleep(retry_delay)
                else:
                    # Append a default value (e.g., None) to the list after max retries
                    judgements.append(None)
            else:
                # If it's a different error, append None and break the loop
                judgements.append(None)
                break

Processing Prompts: 100%|██████████| 1683/1683 [34:13<00:00,  1.22s/it]


In [ ]:
#くっつける
df2.loc[:,'judgement'] = judgements

# Convert 'judgement' to binary: True -> 1, False -> 0 うまく返ってこなかったものは組入扱いにする
df2['judgement'] = df2['judgement'].fillna(1).astype(int)

#フィルタリング
df3 = df2[df2["judgement"] == 1]




<ipython-input-69-0576aafb6e97>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['judgement'] = df2['judgement'].fillna(1).astype(int)


In [ ]:
df3.shape

(578, 22)

In [ ]:
#不要な列を削除
df3.drop(columns=['judgement'], inplace=True)

#df3を500行ずつに分ける
chunks = [df3[i:i+500] for i in range(0, len(df3), 500)]

#それぞれをエクスポート
for i, chunk in enumerate(chunks):
    chunk.to_csv(f'articles_judgement_others_{i}.csv', index=False)

#zipにする
!zip -r articles_judgement_others.zip articles_judgement_others*

#download
from google.colab import files
files.download('articles_judgement_others.zip')

<ipython-input-71-4dae44545b05>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(columns=['judgement'], inplace=True)


  adding: articles_judgement_others_0.csv (deflated 83%)
  adding: articles_judgement_others_1.csv (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>